# Datasets Merging
see [./data/sources.csv](./data/sources.csv) for the different sources where we were able to download each datasets

Simply put there exist a few dataset containing information about the impact of mutations on protein stability.
The main 3 big dataset that exist as of Oct. 2022 are:
- FireProtDB
- ThermoMutDB
- ProThermDB

Which all incorporates data from the same old DB (ProTherm) and additionnal data added by each DB devs.
There exists also other datasets, that were cited in papers from the litterature.
We can expect a lot of redundancies between each datasets.


The goal of this Notebook is to compare the different datasets created in a [former notebook](./individual_datasets_creation.ipynb) and merge them all together.

In [30]:
import os
import glob
import pandas as pd
import numpy as np

COLUMNS = ["PDB_wild", "mutated_chain", "mutation_code", "pH", "Texp", "Tm", "ddG", "dTm"]

Texp_range = [20+273.15, 30+273.15]
# , "mutation_sequence_code", 
features_cols = ["PDB_wild", "mutation_code"]
target_cols = ["Tm", "ddG", "dTm"]


In [31]:
def convert_columns(df):
    df = df.astype({"PDB_wild": object, "mutated_chain": object, 
                    "mutation_code": object,
                    "pH": float, "Tm": float, "ddG": float, 
                    "dTm": float
                    })
    # "mutation_sequence_code": object, "Texp": float,
    return df

In [32]:
# we will start with the fireprotdb datasets, which contains only data with either ddG or dTm
main_df = pd.read_csv("./data/main_dataset/fireprotdb_ddg_dtm_curated.csv")
# for now we want to compare, so we only take the COLUMNS, 
# as they are the same accross all file from ./data/main_dataset

main_df = main_df[COLUMNS]
main_df = convert_columns(main_df)
main_df["dataset_source"] = "FireProtDB"

# This main_df has redundancies ! But we choose to keep those, as they are all from a currated dataset.
# We follow the hypothesis that the people at fireprotDB who manually curated the dataset know how to 
# do it better than we do.
print(len(main_df))
main_df.head()

9805


,PDB_wild,mutated_chain,mutation_code,pH,Tm,ddG,dTm,dataset_source
0,1CQW,A,V245L,NaN,52.5,NaN,2.1,FireProtDB
1,1CQW,A,L95V,NaN,50.0,NaN,-0.4,FireProtDB
2,1CQW,A,C176F,NaN,55.6,NaN,5.2,FireProtDB
3,1CQW,A,G171Q,NaN,53.5,NaN,3.1,FireProtDB
4,1CQW,A,T148L,NaN,51.5,NaN,1.1,FireProtDB


In [33]:
for path in glob.glob("./data/main_dataset/*.csv"):
    if "fireprotdb_ddg_dtm" in path or "main.csv" in path:
        continue
    dataset_src = path.split('/')[-1][:-4]
    print(dataset_src)

    df = pd.read_csv(path)
    df = df[COLUMNS]
    df = convert_columns(df)
    l1, l2 = len(df), len(main_df)

    # Drop rows where we don't have ddG or dTm
    df = df[~(pd.isna(df.ddG) & pd.isna(df.dTm))]
    # Drop rows that are already in main_df by merging and keeping only 'left' (df) values
    unique_val_df = (pd.merge(df,main_df, indicator=True, how='left', on=COLUMNS)
                        .query('_merge=="left_only"')
                        .drop('_merge', axis=1))
    # drops duplicated columns
    unique_val_df = unique_val_df.loc[:,~unique_val_df.columns.duplicated()]
    # adds the dataset source
    unique_val_df["dataset_source"] = dataset_src



    print(f"{l1-len(unique_val_df)}/{l1} rows drop from {dataset_src} as redundancies with the main_df")
    main_df = pd.concat([main_df, unique_val_df], ignore_index=True)
    print(f"added {len(main_df)-l2} from {dataset_src}")

o2567
0/2567 rows drop from o2567 as redundancies with the main_df
added 2567 from o2567
prothermdb
8/9524 rows drop from prothermdb as redundancies with the main_df
added 9516 from prothermdb
ssym
4/704 rows drop from ssym as redundancies with the main_df
added 700 from ssym
thermomut
341/13337 rows drop from thermomut as redundancies with the main_df
added 12996 from thermomut
jinyuan_sun
0/4216 rows drop from jinyuan_sun as redundancies with the main_df
added 4216 from jinyuan_sun
deepddg
0/5720 rows drop from deepddg as redundancies with the main_df
added 5720 from deepddg
kaggle
28981/28981 rows drop from kaggle as redundancies with the main_df
added 0 from kaggle


In [34]:
main_df.to_csv(f"./data/main_dataset/main.csv", index=False)
print(len(main_df))

45520
